In [1]:
import numpy as np
import utils
import pandas as pd

In [2]:
data = pd.DataFrame({"a":[1,2,3], "b":[3,4,5]}, columns=["a","b"])
print(data.head())
print("-------------")
print(data.sum())
print("-------------")
print(np.array(np.array([1,2])+data.sum()))


   a  b
0  1  3
1  2  4
2  3  5
-------------
a     6
b    12
dtype: int64
-------------
[ 7 14]


In [3]:
from itertools import permutations


IMPUTATIONS = ["cca", "mean", "mice_def", "coldel"]  # , "reg"]
IMPUTATION_COMBOS = [perm[0]+perm[1] for perm in permutations(IMPUTATIONS, 2)]

In [4]:
synth_regular = utils.load_synthetic()
compas = utils.load_compas_alt()
RESPONSE = "two_year_recid"

In [5]:
#Checking that the classes don't become too much more imbalanced when data is removed.

#Synthetic data
for key, value in synth_regular.items():
    #print(value.columns)
    print("Key: ", key)
    print("Caucasian = 1: ",len(value[value["is_Caucasian"]==1])/len(value) ,
    "\nCaucasian = 0: ", len(value[value["is_Caucasian"]==0])/len(value))
    print("Male = 1: ",len(value[value["gender_factor"]==1])/len(value) ,
    "\nMale = 0: ", len(value[value["gender_factor"]==0])/len(value))
    print("After 30% missing: ")
    temp = value.copy()
    temp = utils.impute(utils.data_remover_cat(temp, "is_Caucasian", 30), "is_Caucasian")
    print("Caucasian = 1: ",len(temp[temp["is_Caucasian"]==1])/len(temp) ,
        "\nCaucasian = 0: ", len(temp[temp["is_Caucasian"]==0])/len(temp))
    print("Male = 1: ",len(temp[temp["gender_factor"]==1])/len(temp) ,
        "\nMale = 0: ", len(temp[temp["gender_factor"]==0])/len(temp))

Key:  test
Caucasian = 1:  0.36236236236236236 
Caucasian = 0:  0.6376376376376376
Male = 1:  0.8123123123123123 
Male = 0:  0.18768768768768768
After 30% missing: 
Caucasian = 1:  0.3372827804107425 
Caucasian = 0:  0.6627172195892576
Male = 1:  0.7717219589257504 
Male = 0:  0.2282780410742496
Key:  train
Caucasian = 1:  0.33808095952023987 
Caucasian = 0:  0.6619190404797601
Male = 1:  0.8115942028985508 
Male = 0:  0.18840579710144928
After 30% missing: 
Caucasian = 1:  0.3148531237070749 
Caucasian = 0:  0.6851468762929251
Male = 1:  0.7807199007033513 
Male = 0:  0.21928009929664874


In [6]:
#Compas data
"""for ver in ["standard", "violent"]:
    print("Compas dataset type: ", ver)"""
for key, value in compas.items():
    #print(value.columns)
    print("Key: ", key)
    print("Caucasian = 1: ",len(value[value["is_Caucasian"]==1])/len(value) ,
    "\nCaucasian = 0: ", len(value[value["is_Caucasian"]==0])/len(value))
    print("Male = 1: ",len(value[value["gender_factor"]==1])/len(value) ,
    "\nMale = 0: ", len(value[value["gender_factor"]==0])/len(value))
    print("Response: ", len(value[value["two_year_recid"]==1])/len(value))
    print("After 90% missing: ")
    temp = value.copy()
    temp = utils.impute(utils.data_remover_cat(temp, "is_Caucasian", 90), "is_Caucasian")
    print("sum", temp["two_year_recid"].sum())
    print("Caucasian = 1: ",len(temp[temp["is_Caucasian"]==1])/len(temp) ,
        "\nCaucasian = 0: ", len(temp[temp["is_Caucasian"]==0])/len(temp))
    print("Male = 1: ",len(temp[temp["gender_factor"]==1])/len(temp) ,
        "\nMale = 0: ", len(temp[temp["gender_factor"]==0])/len(temp))
    print("Response: ", len(temp[temp["two_year_recid"]==1])/len(temp))


Key:  train
Caucasian = 1:  0.3388149939540508 
Caucasian = 0:  0.6611850060459492
Male = 1:  0.8099153567110037 
Male = 0:  0.19008464328899638
Response:  0.5429262394195888
After 90% missing: 
sum 171
Caucasian = 1:  0.353125 
Caucasian = 0:  0.646875
Male = 1:  0.69375 
Male = 0:  0.30625
Response:  0.534375
Key:  test
Caucasian = 1:  0.3446244477172312 
Caucasian = 0:  0.6553755522827688
Male = 1:  0.8090328915071183 
Male = 0:  0.19096710849288168
Response:  0.5488463426607757
After 90% missing: 
sum 104
Caucasian = 1:  0.3741935483870968 
Caucasian = 0:  0.6258064516129033
Male = 1:  0.7483870967741936 
Male = 0:  0.25161290322580643
Response:  0.6709677419354839


In [7]:
IMPUTATIONS = ["cca", "mean", "mice_def", "coldel"]  # , "reg"]
IMPUTATION_COMBOS = [perm[0]+"|"+perm[1] for perm in permutations(IMPUTATIONS, 2)]

In [8]:
IMPUTATION_COMBOS

['cca|mean',
 'cca|mice_def',
 'cca|coldel',
 'mean|cca',
 'mean|mice_def',
 'mean|coldel',
 'mice_def|cca',
 'mice_def|mean',
 'mice_def|coldel',
 'coldel|cca',
 'coldel|mean',
 'coldel|mice_def']

In [9]:
percentiles = [p for p in range(0,20,2)]+[p for p in range(20,80, 10)]
#TODO test missing = Sensitive in case code crashes
missing=["priors_count"]#, "two_year_recid"]# "is_Caucasian"]#, "gender_factor", "crime_factor"]
all_results = {"Full data": {}, "Averaged results": {} }
RUNS = 2
for miss in missing:
    for sensitive in ["is_Caucasian"]:#, "gender_factor"]:
        #try:
        recid_results = utils.test_bench(data = "compas", pred = RESPONSE, missing = miss, sensitive=sensitive,
                        percentiles = percentiles, n_runs=RUNS)
        #synth_results = utils.test_bench(data = "synthetic", pred = RESPONSE, missing = miss, sensitive=sensitive,
                            #percentiles = percentiles, n_runs=RUNS)
        #TODO and remember to fix data v_recid_results = utils.test_bench(train = compas["standard"]["train"],test = compas["standard"]["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                        #percentiles = percentiles)
        #all_results[miss+"_"+sensitive+"_"+"synth"] = synth_results 
        all_results["Full data"][miss+"_"+sensitive+"_"+"recid"] = recid_results["Full data"]
        all_results["Averaged results"][miss+"_"+sensitive+"_"+"recid"] = recid_results["Averaged results"]
        #except:
            #print("failed at " + sensitive + miss)
        """try:
            synth_results = utils.test_bench(train = synth_regular["train"],test = synth_regular["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                            percentiles = percentiles)
            recid_results = utils.test_bench(train = compas["train"],test = compas["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                            percentiles = percentiles)
            #TODO and remember to fix data v_recid_results = utils.test_bench(train = compas["standard"]["train"],test = compas["standard"]["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                            #percentiles = percentiles)
            all_results[miss+"_"+sensitive+"_"+"synth"] = synth_results 
            all_results[miss+"_"+sensitive+"_"+"recid"] = recid_results 
            #all_results[miss+"_"+sensitive+"_"+"v_recid"] = v_recid_results
        except Exception as e:
            print("Exception: ", e)
            print("Parameters: ", sensitive+"_"+miss) 
        """

100%|██████████| 2/2 [03:47<00:00, 113.82s/it]


TEMP_RESULTSmcarccamean 992
TEMP_RESULTSmcarccamice_def 992
TEMP_RESULTSmcarccacoldel 992
TEMP_RESULTSmcarmeanmice_def 992
TEMP_RESULTSmcarmeancoldel 992
TEMP_RESULTSmcarmice_defcoldel 992
TEMP_RESULTSmarccamean 992
TEMP_RESULTSmarccamice_def 992
TEMP_RESULTSmarccacoldel 992
TEMP_RESULTSmarmeanmice_def 992
TEMP_RESULTSmarmeancoldel 992
TEMP_RESULTSmarmice_defcoldel 992
['spd', 'eo0', 'eo1', 'eosum', 'pp', 'acc', 'tpr0', 'tpr1', 'tnr0', 'tnr1']


<Figure size 1440x792 with 0 Axes>

In [10]:
import json
from pathlib import Path
with open(Path("raw_data/test.json"), 'w') as f:
            json.dump(all_results, f)